In [ ]:
!pip install --upgrade pip

In [ ]:
# %pip install huggingface-hub
# %pip install numpy
# %pip install opencv-python
# %pip install matplotlib
# %pip install pandas

In [ ]:
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

In [ ]:
!git clone https://github.com/taki0112/vit-tensorflow.git

In [ ]:
!mv "/kaggle/working/vit-tensorflow/vit_tensorflow" "/kaggle/working/vit_tensorflow"

In [ ]:
!pip install -U einops

In [ ]:
!pip install optree

## Import

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import os
import json
import re
import time
import math
import numpy as np

In [ ]:
import cv2
import matplotlib.pyplot as plt

## Config

In [ ]:
# secret_hf =

In [ ]:
# !huggingface-cli login --token $secret_hf

## Download data

In [ ]:
!mkdir dataset

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/Hoc-sau-va-ung-dung-nhom-2", filename="imagedata.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)
hf_hub_download(repo_id="hotamago/Hoc-sau-va-ung-dung-nhom-2", filename="label.txt", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)

In [ ]:
!sudo apt-get install unzip

In [ ]:
!mkdir datasetImage
!unzip -q -o "dataset/imagedata.zip" -d "datasetImage"

## Load data

In [ ]:
prefix_path_data = "/kaggle/working/"

In [ ]:
rawlabel = ""

In [ ]:
# load label
with open(os.path.join(prefix_path_data, "dataset/label.txt"), "r") as f:
    rawlabel = f.read()
    
# G93aRj07CIZPKaC8.jpg {"label-8":14,"label-7":3,"label-2":4} 16
# name image, label(json), totel people vote
# Format to json {name, label, totel}
labelDataset = []
arrayRawLabel = rawlabel.split("\n")
for ele in arrayRawLabel:
    eleArr = ele.split(" ")
    try:
        labelDataset.append({
            "name": eleArr[0],
            "label": json.loads(eleArr[1]),
            "total": int(eleArr[2])
        })
    except:
        print(ele)

In [ ]:
# Label 0 -> 10, 0 is not good image need to delete
def LabelsToVector(labels):
    vectorLabel = [0] * 11
    try:
        for label, num in labels.items():
            if len(label.split("-")) != 2:
                print("Error: ", labels)
            vectorLabel[int(label.split("-")[1])] = num
    except:
        pass
    return vectorLabel

def NormalVectorLabel(vectorLabel, total, presentFilter):
    # Check if label 0 is > presentFilter
    if total == 0 or vectorLabel[0]/total > presentFilter:
        vectorLabel[0] = 1
        for i in range(1, len(vectorLabel)):
            vectorLabel[i] = 0
        return np.array(vectorLabel)
    
    # Process localcation label
    totalLocalVote = 0
    for i in range(1, 3):
        totalLocalVote += vectorLabel[i]
    if totalLocalVote > 0:
        for i in range(1, 3):
            vectorLabel[i] = 1 if vectorLabel[i]/totalLocalVote > presentFilter else 0
    
    # Process orther label
    for i in range(3, len(vectorLabel)):
        vectorLabel[i] = 1 if vectorLabel[i]/total > presentFilter else 0
        
    # Process speacil case
    # Auto set label 1
    goodLabel = [3, 4]
    for i in range(len(goodLabel)):
        if vectorLabel[goodLabel[i]] == 1:
            vectorLabel[1] = 1;
    
    # Auto set label 2
    goodLabel = [7, 8]
    for i in range(len(goodLabel)):
        if vectorLabel[goodLabel[i]] == 1:
            vectorLabel[2] = 1;
            
    # Check if label 1 or 2 is set
    if vectorLabel[1] == vectorLabel[2]:
        vectorLabel[0] = 1
        for i in range(1, len(vectorLabel)):
            vectorLabel[i] = 0
        return np.array(vectorLabel)
#     else:
#         vectorLabel[0] = 0
    
    return np.array(vectorLabel)

In [ ]:
# Filter label
for ele in labelDataset:
    # Only get label with > 50% vote
    ele["label"] = NormalVectorLabel(LabelsToVector(ele["label"]), ele["total"], 0.5)
    
# Filter bad image
clearLabelDataset = []
for ele in labelDataset:
    if ele["label"][0] == 0:
        ele["label"] = ele["label"][1:]
        clearLabelDataset.append(ele)
labelDataset = clearLabelDataset

In [ ]:
# Add map image name to label
mapImageNameToLabel = {}
for ele in labelDataset:
    mapImageNameToLabel[ele["name"]] = ele["label"]

In [ ]:
# Map name for fast filter image
mapNameFastQuery = set()
for ele in labelDataset:
    mapNameFastQuery.add(ele["name"])

In [ ]:
# resize image with max(width, height) = size_image_model
def resize_image_reduce_size(image, size_image_model):
    min_size = min(image.shape[0], image.shape[1])
    radio_h = image.shape[0] / min_size
    radio_w = image.shape[1] / min_size
    return cv2.resize(image, (int(radio_w * size_image_model), int(radio_h * size_image_model)))

In [ ]:
# Load image
imagesDataset = {}
for label in labelDataset:
    if label["name"] not in mapNameFastQuery:
        continue
    img = resize_image_reduce_size(
        cv2.imread(os.path.join(prefix_path_data, "datasetImage/file", label['name'])),
        512
    )
    imagesDataset[label['name']] = img

## Process

In [ ]:
print(len(labelDataset))
print(len(imagesDataset))

In [ ]:
# Plot analytic banlance dataset

totalLabelDataset = np.array([0] * 10)
for ele in labelDataset:
    totalLabelDataset += ele["label"]

print(totalLabelDataset)
    
# Draw
plt.bar([i for i in range(1, 11)], totalLabelDataset)

In [ ]:
print(labelDataset[0])

In [ ]:
csvDataJSON = []
listGoodImage = []
labelValue = ["Hồ Gươm", "Hồ Tây", "Tháp rùa", "Cầu Thê Húc", "Bưu Điện", "Vườn Hoa", "Chùa Trấn Quốc", "Đền Quán Thánh", "Khách Sạn", "Công Viên Nước"]
for ele in labelDataset:
    labelTemp = {}
    for i in range(len(labelValue)):
        labelTemp[labelValue[i]] = ele["label"][i]
    csvDataJSON.append({
        "file": ele["name"],
        **labelTemp
    })
    listGoodImage.append(ele["name"])

In [ ]:
# Create csv file
import pandas as pd
pd.DataFrame(csvDataJSON).to_csv("labelData.csv", index=False)

In [ ]:
# Create list good 
f = open("listimage.txt", "w")
f.write("\n".join(listGoodImage))
f.close()

In [ ]:
# Note:
# imagesDataset is an Dict key is name image, value is image data
# Using: get image data of label name N => imagesDataset[N]
# Get image data of label i => imagesDataset[labelDataset[i]["name"]]

## Process data to send

In [ ]:
from functools import cmp_to_key

In [ ]:
!rm -rf compressImage

In [ ]:
!mkdir compressImage

In [ ]:
labelDataset.sort(key=cmp_to_key(lambda x, y: y["total"] - x["total"]))

In [ ]:
listCompressConfig = [100, 80, 50, 30, 20, 10, 5]

In [ ]:
# Get all label type 9
listAddionEle = []
for ele in labelDataset:
    if ele["label"][8] >= 1:
        listAddionEle.append(ele)

print(len(listAddionEle))

# Add image from labelDataset
for ele in labelDataset[0:400 - len(listAddionEle)]:
    listAddionEle.append(ele)
    
print(len(listAddionEle))

In [ ]:
labelValue = ["Hồ Gươm", "Hồ Tây", "Tháp rùa", "Cầu Thê Húc", "Bưu Điện", "Vườn Hoa", "Chùa Trấn Quốc", "Đền Quán Thánh", "Khách Sạn", "Công Viên Nước"]
csvDataJSON = []
totalLabelDataset = np.array([0] * 10)
i = 0
for ele in listAddionEle:
    i += 1
    file_name = os.path.join("compressImage", "{0}.jpg".format(i))
    for j in range(len(listCompressConfig)):
        cv2.imwrite(
            file_name, imagesDataset[ele["name"]],
            [int(cv2.IMWRITE_JPEG_QUALITY), listCompressConfig[j]]
        )
        file_size = os.stat(file_name).st_size / 1024
        if file_size <= 20.0:
            break
    file_size = os.stat(file_name).st_size / 1024
    if file_size > 20.0:
        print("Error outsize {0}".format(i))
        
    # Add analytics
    totalLabelDataset += ele["label"]
    
    # Add csv
    labelTemp = {}
    for j in range(len(labelValue)):
        labelTemp[labelValue[j]] = ele["label"][j]
    csvDataJSON.append({
        "file": "{0}.jpg".format(i),
        **labelTemp
    })
    
# Create csv file
import pandas as pd
pd.DataFrame(csvDataJSON).to_csv("labelData.csv", index=False)
    
# Draw
print(totalLabelDataset)
plt.bar([i for i in range(1, 11)], totalLabelDataset)

In [ ]:
!zip -r compressImage.zip compressImage

## Preprocess data

### Analytics data

In [ ]:
minLabel = np.amin(totalLabelDataset)
print(minLabel)

In [ ]:
for ele in labelDataset:
    print(ele)
    break

In [ ]:
def label2str(label):
    s = ""
    for i in range(len(label)):
        s += str(int(label[i]))
    return s
def str2label(s):
    label = []
    for i in range(len(s)):
        label.append(int(s[i]))
    return label

In [ ]:
tempCnumDimOnehot = {}
for ele in labelDataset:
    tempCnumDimOnehot[label2str(ele["label"])] = 1
numDimOnehot = len(tempCnumDimOnehot)
print(numDimOnehot)

In [ ]:
def index2onehot(index):
    onehot = [0] * numDimOnehot
    onehot[index] = 1
    return onehot

In [ ]:
mapLabel2OneHot = {}
for ele in labelDataset:
    if label2str(ele["label"]) not in mapLabel2OneHot:
        mapLabel2OneHot[label2str(ele["label"])] = index2onehot(len(mapLabel2OneHot))

In [ ]:
mapLabel2OneHot

In [ ]:
mapOneHot2Label = {}
for i in mapLabel2OneHot:
    mapOneHot2Label[label2str(mapLabel2OneHot[i])] = str2label(i)

In [ ]:
mapOneHot2Label

In [ ]:
# for ele in labelDataset:
#     ele["label"] = mapLabel2OneHot[label2str(ele["label"])]

In [ ]:
def autoLabel2Onehot(label):
    return mapLabel2OneHot[label2str(label)]
def autoOnehot2Label(onehot):
    return mapOneHot2Label[label2str(onehot)]

In [ ]:
print(autoOnehot2Label(autoLabel2Onehot([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])))

In [ ]:
mapLabel2OneHot = {
    '0100000100': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1010100000': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1010000000': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1001000000': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1000110000': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1000010000': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '0100001000': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1000100000': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '1010010000': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '0100000000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '0100000001': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    '0100000010': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    '0100010000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    '1000000010': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    '1001010000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    '1000000000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    '1011010000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    '1011000000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    '1000000001': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
}
mapOneHot2Label = {
    '1000000000000000000': [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    '0100000000000000000': [1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
    '0010000000000000000': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    '0001000000000000000': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    '0000100000000000000': [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    '0000010000000000000': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    '0000001000000000000': [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    '0000000100000000000': [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    '0000000010000000000': [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    '0000000001000000000': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    '0000000000100000000': [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
    '0000000000010000000': [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
    '0000000000001000000': [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    '0000000000000100000': [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    '0000000000000010000': [1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
    '0000000000000001000': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '0000000000000000100': [1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
    '0000000000000000010': [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
    '0000000000000000001': [1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
}

### Process image

#### Constant

In [ ]:
size_image_model = 224 # 224

#### Helper function

In [ ]:
def crop2nimage(image, nimage = 3):
    # Size of bigest square image
    sizeSquare = min(image.shape[0], image.shape[1])

    # Rotate image if image is portrait
    isRotate = False
    if image.shape[0] > image.shape[1]:
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        isRotate = True

    # Crop n image
    cropImages = []
    widthBetween = (image.shape[1] - sizeSquare) / (nimage - 1)
    for i in range(nimage):
        x1 = int(i * widthBetween)
        x2 = int(x1 + sizeSquare)
        cropImages.append(image[:, x1:x2])

    # Rotate image back
    if isRotate:
        for i in range(len(cropImages)):
            cropImages[i] = cv2.rotate(cropImages[i], cv2.ROTATE_90_COUNTERCLOCKWISE)

    # Convert color to RGB
    for i in range(len(cropImages)):
        cropImages[i] = cv2.cvtColor(cropImages[i], cv2.COLOR_BGR2RGB)
        
    return cropImages

In [ ]:
def calibrate_image(image, target_size_kb=20):
    # Resize image to the target size
    resized_image = cv2.resize(image, (224, 224))

    return resized_image

In [ ]:
# Data augmentation by type:
# 1. Random flip
# 2. Random rotate
# 4. Random Saturate
# 3. Random brightness
# 5. Random crop

# Random flip
def random_flip(image, p=0.5):
    if np.random.rand() < p:
        return cv2.flip(image, 1)
    return image

# Random rotate from range angle
def rotate_image(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    """

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result

def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )

def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

def random_rotate(image, angle_range=20):
    angle = np.random.uniform(-angle_range, angle_range)
    image_height, image_width = image.shape[0:2]
    image = crop_around_center(
        rotate_image(image, angle),
        *largest_rotated_rect(
            image_width,
            image_height,
            math.radians(angle)
        )
    )
    return image

# Random saturate
def random_saturate(image, low=0.5, high=1.5):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 1] = hsv[:, :, 1] * np.random.uniform(low, high)
    hsv[:, :, 1][hsv[:, :, 1] > 255] = 255
    hsv[:, :, 2] = hsv[:, :, 2] * np.random.uniform(low, high)
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Random brightness
def random_brightness(image, low=0.3, high=1.5):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * np.random.uniform(low, high)
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Random crop
def random_crop_size(image, size=224):
    x = np.random.randint(0, image.shape[1] - size)
    y = np.random.randint(0, image.shape[0] - size)
    return image[y:y+size, x:x+size]

# Random augment
RandomAugmentClass = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    brightness_range=[0.3,1.8],
    zoom_range=[0.5,1.0],
)

def random_augment(image):
    image = np.copy(image)
#     image = random_flip(image)
#     image = random_rotate(image)
#     image = random_saturate(image)
#     image = random_brightness(image)
#     image = random_crop_size(image, size_image_model)
#     image = tf.image.random_crop(image, (size_image_model, size_image_model, 3)).numpy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = RandomAugmentClass.random_transform(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

In [ ]:
# Random noise
def random_noise(image, present=0.05):
    # Randome noice range [0, 255] with present 0.05
    lran = int(-255 * present)
    rran = int(255 * present)
    noise = np.random.randint(lran, rran, image.shape)

    # Add noise to image
    image = cv2.add(image, noise, dtype=cv2.CV_8UC3)

    return image

# Random blur
def random_blur(image, win_size=(3, 3)):
    image = cv2.GaussianBlur(image, win_size, 0)
    return image

In [ ]:
# torchvision.transforms.Normalize (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
def torchNormalize(data: np.ndarray, mean: list[float], std: list[float], inplace=False) -> np.ndarray:
    # Inplace operation check
    if not inplace:
        data = np.copy(data).astype("float32")

    # Convert mean and std to NumPy arrays with appropriate shape
    mean = np.asarray(mean, dtype=data.dtype)
    std = np.asarray(std, dtype=data.dtype)

    # Perform normalization
    return (data - data.mean()) / data.std() * std + mean

In [ ]:
def hota_preprocess_input(x):
#     return tf.keras.applications.efficientnet_v2.preprocess_input(np.copy(x))
#     return tf.keras.applications.resnet_v2.preprocess_input(np.copy(x))
    return tf.image.per_image_standardization(x).numpy()
#     return torchNormalize(
#         x,
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
#     return x

#### Resize image

In [ ]:
# resize image with max(width, height) = size_image_model
def resize_image_reduce_size(image, size_image_model):
    min_size = min(image.shape[0], image.shape[1])
    radio_h = image.shape[0] / min_size
    radio_w = image.shape[1] / min_size
    return cv2.resize(image, (int(radio_w * size_image_model), int(radio_h * size_image_model)))

In [ ]:
# # Apply to all image in dataset
for imageId in imagesDataset:
    imagesDataset[imageId] = resize_image_reduce_size(imagesDataset[imageId], size_image_model)

In [ ]:
# cv2.imwrite("test.jpg", imagesDataset[labelDataset[0]["name"]], [cv2.IMWRITE_JPEG_QUALITY, 50])
# print(os.path.getsize('test.jpg')/1024)

#### Data augment

In [ ]:
# Split image name to label
# tempImageLabelName = [[] for _ in range(numDimOnehot)]
# for ele in labelDataset:
#     for i in range(len(mapLabel2OneHot[label2str(ele["label"])])):
#         if mapLabel2OneHot[label2str(ele["label"])][i] >= 1:
#             tempImageLabelName[i].append(ele["name"])

tempImageLabelName = [[] for _ in range(10)]
for ele in labelDataset:
# for ele in listAddionEle:
    for i in range(len(ele["label"])):
        if ele["label"][i] >= 1:
            tempImageLabelName[i].append(ele["name"])

In [ ]:
# total len
totalTempImageLabelName = []
for i in range(len(tempImageLabelName)):
    totalTempImageLabelName.append(len(tempImageLabelName[i]))

In [ ]:
print(np.amax(totalTempImageLabelName), np.amin(totalTempImageLabelName))

In [ ]:
len([i for i in totalTempImageLabelName if i > 0])

In [ ]:
# Target number image each label after augment
target_number_image = 1000

imagesDatasetAug = []
labelDatasetAug = []
# Augment image
for i in range(0, len(tempImageLabelName)):
    print("Augment label: ", i)
    print("Number image: ", len(tempImageLabelName[i]))
    if totalTempImageLabelName[i] >= target_number_image or totalTempImageLabelName[i] == 0:
        continue
        
    numDeltaAdd = target_number_image - totalTempImageLabelName[i]
    for j in range(numDeltaAdd):
        # Name image
        imageName = tempImageLabelName[i][np.random.randint(0, len(tempImageLabelName[i]))]
        # Random select image
        image = imagesDataset[imageName]
        image = random_augment(image)
        imagesDatasetAug.append(image)
        
#         labelDatasetAug.append(autoLabel2Onehot(mapImageNameToLabel[imageName]))
#         for k in range(len(autoLabel2Onehot(mapImageNameToLabel[imageName]))):
#             if autoLabel2Onehot(mapImageNameToLabel[imageName])[k] >= 1:
#                 totalTempImageLabelName[k] += 1
        
        labelDatasetAug.append(mapImageNameToLabel[imageName])
        for k in range(len(mapImageNameToLabel[imageName])):
            if mapImageNameToLabel[imageName][k] >= 1:
                totalTempImageLabelName[k] += 1
        
#         totalTempImageLabelName[i] += 1

In [ ]:
print(totalTempImageLabelName)

In [ ]:
print(len(imagesDatasetAug), len(labelDatasetAug))

In [ ]:
plt.imshow(imagesDatasetAug[124].astype("uint8"))
plt.show()

In [ ]:
print(labelDatasetAug[124])

In [ ]:
# imagesDatasetArray
imagesDatasetArray = []
labelDatasetArray = []

for ele in labelDataset:
# for ele in listAddionEle:
    imagesDatasetArray.append(random_augment(imagesDataset[ele["name"]]))
    labelDatasetArray.append(ele["label"])
#     labelDatasetArray.append(autoLabel2Onehot(ele["label"]))

for i in range(len(imagesDatasetAug)):
    imagesDatasetArray.append(imagesDatasetAug[i])
    labelDatasetArray.append(labelDatasetAug[i])

In [ ]:
print(len(imagesDatasetArray), len(labelDatasetArray))

In [ ]:
import random
# suffer
zipImageDataset = list(zip(imagesDatasetArray, labelDatasetArray))
random.shuffle(zipImageDataset)
imagesDatasetArray, labelDatasetArray = zip(*zipImageDataset)
imagesDatasetArray = list(imagesDatasetArray)
labelDatasetArray = list(labelDatasetArray)

#### Resize image and add some noise and rotate

In [ ]:
imagesDatasetArray3 = []

In [ ]:
for i in range(len(labelDatasetArray)):
    image = np.copy(imagesDatasetArray[i])
    
    # Crop
    cropImages = crop2nimage(image, 3)

    # Resize
    for i in range(len(cropImages)):
        cropImages[i] = cv2.resize(cropImages[i], (size_image_model, size_image_model))

    # Add blur to image
    for i in range(len(cropImages)):
        cropImages[i] = random_blur(cropImages[i], (3, 3))
        
    # Add noise to image
    for i in range(len(cropImages)):
        # Randome noice
        cropImages[i] = random_noise(cropImages[i], 0.1)

    # Debug
#     for i in range(len(cropImages)):
#         plt.imshow(cropImages[i])
#         plt.show()
#     break

    imagesDatasetArray3.append([cropImages[1]])

In [ ]:
print(len(imagesDatasetArray3), len(imagesDatasetArray3[0]))

## Training model

### Normal image

### Import library

### Split testcase

In [ ]:
imagesDatasetArray3_training = []
labelDatasetArray_training = []
imagesDatasetArray3_testing = []
labelDatasetArray_testing = []

In [ ]:
idx = [i for i in range(len(imagesDatasetArray3))]
np.random.shuffle(idx)

for i in range(len(idx)):
    if i < len(idx) * 0.3:
        imagesDatasetArray3_training.append(imagesDatasetArray3[idx[i]])
        labelDatasetArray_training.append(labelDatasetArray[idx[i]])
    else:
        imagesDatasetArray3_testing.append(imagesDatasetArray3[idx[i]])
        labelDatasetArray_testing.append(labelDatasetArray[idx[i]])

In [ ]:
# Convert to numpy
for i in range(len(imagesDatasetArray3_training)):
    imagesDatasetArray3_training[i] = [
        hota_preprocess_input(np.array(imagesDatasetArray3_training[i][j], dtype=np.float32)) for j in range(len(imagesDatasetArray3_training[i]))
    ]
    labelDatasetArray_training[i] = np.array(labelDatasetArray_training[i], dtype=np.float32)
    
for i in range(len(imagesDatasetArray3_testing)):
    imagesDatasetArray3_testing[i] = [
        hota_preprocess_input(np.array(imagesDatasetArray3_testing[i][j], dtype=np.float32)) for j in range(len(imagesDatasetArray3_testing[i]))
    ]
    labelDatasetArray_testing[i] = np.array(labelDatasetArray_testing[i], dtype=np.float32)

# imagesDatasetArray3_training = np.array(imagesDatasetArray3_training, dtype=np.float32) / 255.0
# labelDatasetArray_training = np.array(labelDatasetArray_training, dtype=np.float32)
# imagesDatasetArray3_testing = np.array(imagesDatasetArray3_testing, dtype=np.float32) / 255.0
# labelDatasetArray_testing = np.array(labelDatasetArray_testing, dtype=np.float32)

### Build dataset with batch

In [ ]:
def create_batch_from_arr(arrData, batch_size=64):
    temp_shape = list(arrData.shape)
    deltaMiss = ((arrData.shape[0] + batch_size - 1) // batch_size) * batch_size - arrData.shape[0]
    temp_shape[0] += deltaMiss
    batchArrData = np.zeros(tuple(temp_shape), dtype=np.float32)
    # Add to last
    batchArrData[:arrData.shape[0]] = arrData
    for i in range(arrData.shape[0], batchArrData.shape[0]):
        batchArrData[i] = arrData[i - arrData.shape[0]]
        
    batchArrData = batchArrData.reshape(tuple([-1, batch_size, *list(batchArrData.shape[1:])]))
    
    return batchArrData

def create_dataset(images, labels, batch_size=64):
    if batch_size == 0:
        return (np.array(images, dtype=np.float32), np.array(labels, dtype=np.float32))
    images = np.array(images, dtype=np.float32)
    # image is 3 image in array
    tempImageSplit = []
    for i in range(images.shape[1]):
        tempImageSplit.append(create_batch_from_arr(images[:, i], batch_size))

    imagesBatch = []
    for i in range(len(tempImageSplit[0])):
        nimage = []
        for j in range(len(tempImageSplit)):
            nimage.append(tempImageSplit[j][i])
        imagesBatch.append(nimage)
        
    labels = np.array(labels, dtype=np.float32)
    labels = create_batch_from_arr(labels, batch_size)
    
    return (np.array(imagesBatch), labels)

In [ ]:
datasetTraining = create_dataset(imagesDatasetArray3_training, labelDatasetArray_training, 0)
datasetTesting = create_dataset(imagesDatasetArray3_testing, labelDatasetArray_testing, 0)

In [ ]:
print(np.amin(datasetTraining[0][2][0]), np.amax(datasetTraining[0][2][0]))
print(np.mean(datasetTraining[0][2][0]), np.std(datasetTraining[0][2][0]))
print(datasetTraining[1][0])

### Init model

In [ ]:
# Using dataset imagesDatasetArray3 and labelDatasetArray

In [ ]:
from vit_tensorflow.vit import Transformer
import tensorflow.keras.layers as nn
from tensorflow import einsum
from einops import rearrange, repeat
from einops.layers.tensorflow import Rearrange

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

In [ ]:
class ViT(tf.keras.layers.Layer):
    def __init__(self, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim,
                 pool='cls', dim_head=64, dropout=0.0, emb_dropout=0.0):
        super(ViT, self).__init__()

        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'
        
        self.trearragnge = Rearrange('b (h p1) (w p2) c -> b (h w) (p1 p2 c)', p1=patch_height, p2=patch_width)
        self.patch_embedding = tf.keras.Sequential([
            nn.Dense(units=dim)
        ], name='patch_embedding')

        self.pos_embedding = tf.Variable(initial_value=tf.random.normal([1, num_patches + 1, dim]))
        self.cls_token = tf.Variable(initial_value=tf.random.normal([1, dim]))
        self.dropout = nn.Dropout(rate=emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        
        self.num_classes = num_classes

        self.mlp_head = tf.keras.Sequential([
            nn.LayerNormalization(),
            nn.Dense(units=num_classes)
        ], name='mlp_head')
        
    def build(self, input_shape):
        super(ViT, self).build(input_shape)  # Be sure to call this at the end

    def call(self, img, training=True, **kwargs):
        x = self.trearragnge(img)
        x = self.patch_embedding(x)
        b, n, d = x.shape
        
        cls_tokens = repeat(self.cls_token, 'n d -> b n d', b=b)
        x = tf.concat([cls_tokens, x], axis=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x, training=training)

        x = self.transformer(x, training=training)

        if self.pool == 'mean':
            x = tf.reduce_mean(x, axis=1)
        else:
            x = x[:, 0]

        x = self.mlp_head(x)

        return x
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_classes)

In [ ]:
def hotaConv2D(*arg, **kwarg):
    return tf.keras.layers.Conv2D(*arg, **kwarg) # , kernel_initializer="he_uniform"
def hotaDense(*arg, **kwarg):
    return tf.keras.layers.Dense(*arg, **kwarg)

In [ ]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = hotaConv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x) # axis=3
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = hotaConv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
#     x = hotaConv2D(filter, (1,1), padding = 'same', strides = (2,2))(x)
    return x
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = hotaConv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = hotaConv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    # Processing Residue with conv(1,1)
    x_skip = hotaConv2D(filter, (1,1), strides = (2,2))(x_skip)
#     x_skip = tf.keras.layers.BatchNormalization()(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
#     x = hotaConv2D(filter, (1,1), padding = 'same', strides = (2,2))(x)
    return x
def ResNet(shape = (224, 224, 3), block_layers = [3, 4, 6, 3], using_top_layer = True, classes = 10, activation_class="softmax"):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = hotaConv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
#     x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = hotaDense(512, activation = 'relu')(x)
    if using_top_layer:
#         if activation_class == "sigmoid":
#             x = [hotaDense(1, activation = activation_class)(x) for i in range(classes)]
#         else:
        x = hotaDense(classes, activation = activation_class)(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet12")
    return model

In [ ]:
def hotaBlockImage(index):
    # Build model flow
#     inputTensor = tf.keras.Input(shape=(size_image_model,size_image_model,3))
#     model = tf.keras.applications.EfficientNetV2S(
#         include_top=True,
#         weights=None, # 'imagenet'
#         input_tensor=inputTensor,
#         input_shape=None,
#         pooling=None,
#         classes=512,
#         classifier_activation='relu', # softmax
#         include_preprocessing=False
#     )

#     # Rename layer
#     layer_names=[layer.name for layer in model.layers]
#     for layer_name in layer_names:
#         model.get_layer(name=layer_name).name = "{0}_{1}".format(layer_name, index)

#     return model
    
#     model = ViT(
#         image_size = (size_image_model, size_image_model),
#         patch_size = 32,
#         num_classes = 1000,
#         dim = 1024,
#         depth = 6,
#         heads = 16,
#         mlp_dim = 2048,
#         dropout = 0.1,
#         emb_dropout = 0.1
#     )(inputTensor)
        
#     return tf.keras.Model(inputs=inputTensor, outputs=model)
#     return ResNet(
#         shape = (128, 128, 3), block_layers = [3, 3, 3, 3], using_top_layer = False, classes = 10, activation_class="softmax"
#     )
    pass
def hotaNBlockImage(nblock = 3):
#     inputs = []
#     hidden_outputs = []
#     for i in range(nblock):
#         temp_model = hotaBlockImage(i)
# #         inputs.append(temp_model.input[0]) # Application
#         inputs.append(temp_model.input)
#         hidden_outputs.append(temp_model.output)
        
#     combined = tf.keras.layers.Concatenate()(hidden_outputs)
#     outputflatten = tf.keras.layers.Flatten()(combined)
    
#     hidden_outputs_2 = hotaDense(512, activation="relu")(outputflatten)
# #     outputs = hotaDense(numDimOnehot, activation="softmax")(hidden_outputs_2)
#     outputs = hotaDense(10, activation="sigmoid")(hidden_outputs_2)
    
#     return tf.keras.Model(inputs=inputs, outputs=outputs)
#     return hotaBlockImage(0)

#     return ResNet(
#         shape = (size_image_model, size_image_model, 3),
#         block_layers = [3, 4, 6, 3],
#         using_top_layer = True,
#         classes = 10,
#         activation_class="sigmoid"
#     )
#     inputTensor = tf.keras.Input(shape=(size_image_model,size_image_model,3))
#     return tf.keras.applications.EfficientNetV2S(
#         include_top=True,
#         weights=None, # 'imagenet'
#         input_tensor=inputTensor,
#         input_shape=None,
#         pooling=None,
#         classes=10, # 19
#         classifier_activation='sigmoid', # softmax
#         include_preprocessing=False
#     )

#     inputTensor = tf.keras.Input(shape=(size_image_model,size_image_model,3))
#     return tf.keras.applications.ResNet50V2(
#         include_top=True,
#         weights=None,
#         input_tensor=inputTensor,
#         input_shape=None,
#         pooling=None,
#         classes=10,
#         classifier_activation='sigmoid'
#     )

    inputTensor = tf.keras.Input(shape=(size_image_model,size_image_model,3))
    return tf.keras.applications.EfficientNetB0(
        include_top=True,
        weights=None,
        input_tensor=inputTensor,
        input_shape=None,
        pooling=None,
        classes=10,
        classifier_activation='sigmoid'
    )

In [ ]:
def autoFormatNumpy2List(data):
    listData = []
    for i in range(data.shape[0]):
        listData.append(data[i])
        
    return listData

In [ ]:
def autoFormatListMap2MapList(data):
    listData = []
    for i in range(data.shape[1]):
        listData.append(data[:, i])
    return listData

### GPU Training

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

In [ ]:
hotaModel = hotaNBlockImage(3)

In [ ]:
hotaModel.summary()

In [ ]:
# tf.keras.utils.plot_model(hotaModel, show_shapes=True, expand_nested=True, dpi=40)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
# CategoricalCrossentropy BinaryCrossentropy BinaryFocalCrossentropy
"""
BinaryFocalCrossentropy(
    apply_class_balancing=True,
    alpha=0.92,
    gamma=2.0,
)
BinaryCrossentropy(reduction="sum")
"""
# lossFc = [tf.keras.losses.BinaryCrossentropy() for _ in range(10)]
lossFc = tf.keras.losses.BinaryCrossentropy()
listMetrics = [
    tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="accuracy"), # BinaryAccuracy CategoricalAccuracy
    tf.keras.metrics.AUC(multi_label=True, num_labels=10, name="auc"),
    tf.keras.metrics.F1Score(average="micro", threshold=0.5, name="f1"),
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
hotaModel.compile(loss=lossFc, optimizer=opt, metrics=listMetrics)

In [ ]:
xTrainignInput = autoFormatListMap2MapList(datasetTraining[0])[0]

In [ ]:
# datasetTrainingYSigmoid = [list(datasetTraining[1][i]) for i in range(len(datasetTraining[1]))]
# datasetTrainingYSigmoid = list(zip(*datasetTrainingYSigmoid))
# for i in range(len(datasetTrainingYSigmoid)):
#     datasetTrainingYSigmoid[i] = np.array(datasetTrainingYSigmoid[i], dtype="float32").reshape(-1, 1)

In [ ]:
callbackEarlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-4, patience=3)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="saved_checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    monitor='val_auc', # val_auc val_accuracy
    mode='max',
    save_best_only=True
)
historyTrain = hotaModel.fit(
    x=xTrainignInput,
    y=datasetTraining[1],
    batch_size=24,
    epochs=300,
    callbacks=[callbackEarlyStop, model_checkpoint_callback],
    verbose=1,
    validation_split=0.2,
    shuffle=True,
    initial_epoch=0,
#     class_weight={0: 1, 1: 10}
)

In [ ]:
plt.plot(historyTrain.history["loss"])
plt.plot(historyTrain.history["val_loss"])
plt.show()
plt.plot(historyTrain.history["accuracy"])
plt.plot(historyTrain.history["val_accuracy"])
plt.show()
plt.plot(historyTrain.history["f1"])
# plt.show()
plt.plot(historyTrain.history["val_f1"])
plt.show()
plt.plot(historyTrain.history["auc"])
plt.plot(historyTrain.history["val_auc"])
plt.show()

In [ ]:
xTestingInput = autoFormatListMap2MapList(datasetTesting[0])

In [ ]:
hotaModel.load_weights("saved_checkpoint/checkpoint.weights.h5")

In [ ]:
hotaModel.evaluate(
    x=xTestingInput,
    y=datasetTesting[1],
    batch_size=64,
    verbose=1,
)

### TPU Training

In [ ]:
try: # detect TPUs
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    
    print('Running on TPU ', resolver.master())
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
with strategy.scope():
    hotaModel = hotaNBlockImage(3)

In [ ]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
#     opt = tf.keras.optimizers.SGD(learning_rate=1e-1)
    # CategoricalCrossentropy BinaryCrossentropy BinaryFocalCrossentropy
    lossFc = tf.keras.losses.CategoricalCrossentropy()
    hotaModel.compile(loss=lossFc, optimizer=opt, metrics=[
        # BinaryAccuracy CategoricalAccuracy
        tf.keras.metrics.CategoricalAccuracy("accuracy"),
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.F1Score(name="f1"),
    ])

In [ ]:
hotaModel.summary()

In [ ]:
xTrainignInput = autoFormatListMap2MapList(datasetTraining[0])
xTestingInput = autoFormatListMap2MapList(datasetTesting[0])

In [ ]:
batch_size = 4 * 8

In [ ]:
steps_per_epoch = int(xTrainignInput[0].shape[0] * 0.8)//batch_size
validation_steps = int(xTrainignInput[0].shape[0] * 0.2)//batch_size

In [ ]:
callbackEarlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-4, patience=3)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="saved_checkpoint/checkpoint.keras",
    save_weights_only=False,
    monitor='val_accuracy', # val_auc val_accuracy
    mode='max',
    save_best_only=True
)
historyTrain = hotaModel.fit(
    x=xTrainignInput,
    y=datasetTraining[1],
    batch_size=batch_size,
    epochs=300,
    callbacks=[
        callbackEarlyStop,
        model_checkpoint_callback
    ],
    verbose=1,
    validation_split=0.2,
    shuffle=True,
#     initial_epoch=0,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

In [ ]:
plt.plot(historyTrain.history["accuracy"])
plt.plot(historyTrain.history["val_accuracy"])
plt.show()
plt.plot(historyTrain.history["f1"])
plt.plot(historyTrain.history["val_f1"])
plt.show()
plt.plot(historyTrain.history["auc"])
plt.plot(historyTrain.history["val_auc"])
plt.show()

In [ ]:
hotaModel = tf.keras.models.load_model("saved_checkpoint/checkpoint.keras")

In [ ]:
hotaModel.evaluate(
    x=xTestingInput,
    y=datasetTesting[1],
    batch_size=batch_size,
    verbose=1,
)

### Test predit model

In [ ]:
def autoFormatInput2PreditOutput(data):
    return [data[i].astype("float32").reshape((1, size_image_model, size_image_model, 3)) for i in range(len(data))]

def predict2binary(result):
    for i in range(len(result)):
        for j in range(len(result[i])):
            result[i][j] = 1 if result[i][j] >= 0.5 else 0
    return result

def formatImageInput(image, type_format = 0):
    for i in range(len(image)):
        image[i] = np.array(image[i], dtype=np.float32)
        if type_format == 1:
            image[i] = image[i] / np.float32(255.0)
        elif type_format == 2:
            image[i] = image[i] / np.float32(255.0)
            image[i] = image[i] - np.float32(0.5)
            image[i] = image[i] / np.float32(0.5)
        elif type_format == 3:
            image[i] = hota_preprocess_input(image[i])
    return image

In [ ]:
testcase = 131
for i in range(len(imagesDatasetArray3[testcase])):
    plt.imshow(imagesDatasetArray3[testcase][i])
    plt.show()
print(predict2binary(hotaModel.predict(
    autoFormatInput2PreditOutput(
        hota_preprocess_input(imagesDatasetArray3[testcase])
    ),
    batch_size=1
)))
print(labelDatasetArray[testcase])

### Save model

In [ ]:
!mkdir saved_model

In [ ]:
model_name = "hotaEfficientNetB0_sigmoid_128x128_v6.keras"

In [ ]:
hotaModel.save(os.path.join("saved_model", model_name))

In [ ]:
# Only use for TPU
with strategy.scope():
    hotaModel.save(os.path.join("saved_model", model_name))

In [ ]:
testModel = hotaModel

#### Upload to huggingface

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf_write = user_secrets.get_secret("HUGGINGFACE_WRITE_TOKEN")

In [ ]:
!huggingface-cli login --token $secret_hf_write

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj=os.path.join("/kaggle/working/saved_model", model_name),
    path_in_repo=model_name,
    repo_id="hotamago/deep-learning-and-application-group-02",
    repo_type="model",
)

### Load and testing model

In [ ]:
model_name = "hotaResnet50V2_sigmoid_128x128_v6.keras"

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/deep-learning-and-application-group-02", filename=model_name, revision="main", repo_type="model", local_dir="saved_model", local_dir_use_symlinks=False)

In [ ]:
testModel = tf.keras.models.load_model(os.path.join('saved_model', model_name))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
testModel.compile(
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits = False,
    ),
    optimizer=opt,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.F1Score(),
    ]
)

In [ ]:
xTrainignInput = autoFormatListMap2MapList(datasetTraining[0])
xTestingInput = autoFormatListMap2MapList(datasetTesting[0])

In [ ]:
testModel.evaluate(
    x=xTestingInput,
    y=datasetTesting[1],
    batch_size=64,
    verbose=1,
)

In [ ]:
def autoFormatInput2PreditOutput(data):
    return [data[i].reshape((1, 224, 224, 3)) for i in range(len(data))]

def predict2binary(result):
    for i in range(len(result)):
        for j in range(len(result[i])):
            result[i][j] = 1 if result[i][j] >= 0.5 else 0
    return result

def formatImageInput(image, type_format = 0):
    for i in range(len(image)):
        image[i] = np.array(image[i], dtype=np.float32)
        if type_format == 1:
            image[i] = image[i] / np.float32(255.0)
        elif type_format == 2:
            image[i] = image[i] / np.float32(255.0)
            image[i] = image[i] - np.float32(0.5)
            image[i] = image[i] / np.float32(0.5)
    return image
def formatSoftmaxOutput(label):
    for bat in range(len(label)):
        indexOne = np.argmax(label[bat])
        for i in range(len(label[bat])):
            label[bat][i] = 0
        label[bat][indexOne] = 1
    return label

In [ ]:
testcase = 874
# for i in range(len(imagesDatasetArray3[testcase])):
#     plt.imshow(imagesDatasetArray3[testcase][i])
#     plt.show()
plt.imshow(imagesDatasetArray3[testcase][1])
plt.show()
print(autoOnehot2Label(formatSoftmaxOutput(
    testModel.predict(
        autoFormatInput2PreditOutput(
            formatImageInput(np.copy(imagesDatasetArray3[testcase]).astype('float32'), 2)
        ),
        batch_size=1
    )
)[0]))
print(labelDatasetArray[testcase])

### Test convert for softmax

In [ ]:
datasetTraining = create_dataset(imagesDatasetArray3_training, labelDatasetArray_training, 64)
datasetTesting = create_dataset(imagesDatasetArray3_testing, labelDatasetArray_testing, 64)

In [ ]:
hotaMetrics = [
    tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="accuracy"), # BinaryAccuracy CategoricalAccuracy
    tf.keras.metrics.AUC(multi_label=True, num_labels=10, name="auc"),
    tf.keras.metrics.F1Score(average="micro", threshold=0.5, name="f1"),
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]

In [ ]:
def convertFirst2List(image):
    return [i for i in image]
def autoOnehot2LabelBatch(onehots):
    labels = []
    for i in range(len(onehots)):
        labels.append(autoOnehot2Label(onehots[i]))
    return labels

In [ ]:
dataRunSoft = datasetTesting # datasetTesting datasetTraining
for j in range(len(hotaMetrics)):
    hotaMetrics[j].reset_state()
for i in range(len(dataRunSoft[0])):
    testcaseImage = dataRunSoft[0][i]
    testcaseLabel = autoOnehot2LabelBatch(np.copy(dataRunSoft[1][i]).astype('float32'))
    
    predictOut = np.array(autoOnehot2LabelBatch(formatSoftmaxOutput(
        testModel.predict(
            convertFirst2List(np.copy(testcaseImage).astype('float32')),
            batch_size=64
        )
    )), dtype=np.float32)
    
    # update metrics
    for j in range(len(hotaMetrics)):
        hotaMetrics[j].update_state(testcaseLabel, predictOut)
    
    # Print
    print("Done {0} / {1} batch".format(i + 1, len(dataRunSoft[0])))

In [ ]:
for j in range(len(hotaMetrics)):
    print("{0}: {1}".format(hotaMetrics[j].name, hotaMetrics[j].result()))

### Fine tuning

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-4, patience=5)
testModel.fit(
    x=xTrainignInput,
    y=datasetTraining[1],
    batch_size=8,
    epochs=100,
    callbacks=[callback],
    verbose=1,
    validation_split=0.3,
    shuffle=True,
    initial_epoch=0,
)

## Run final testset

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(
    repo_id="hotamago/Hoc-sau-va-ung-dung-nhom-2",
    filename="TestSet.zip",
    revision="main",
    repo_type="dataset",
    local_dir="dataset",
    local_dir_use_symlinks=False
)

In [ ]:
!mkdir testsetImage
!unzip -q -o "dataset/TestSet.zip" -d "testsetImage"

In [ ]:
# load label
with open(os.path.join("testsetImage","Images","test.txt"), "r") as f:
    rawlabel = f.read()

In [ ]:
labelStruct = rawlabel.split("\n")

In [ ]:
labelOnlyStruct = labelStruct[12:-2]
for i in range(len(labelOnlyStruct)):
    labelOnlyStruct[i] = labelOnlyStruct[i].split(",")

In [ ]:
shapeList = []
for label in labelOnlyStruct:
    img = cv2.imread(os.path.join("testsetImage", "Images", label[0]))
    shapeList.append(np.array(img.shape, dtype="int16"))
shapeList = np.array(shapeList)

In [ ]:
print(np.min(shapeList[:, 0]), np.min(shapeList[:, 1]))
print(np.max(shapeList[:, 0]), np.max(shapeList[:, 1]))

In [ ]:
# Load image
imagesTestset = {}
for label in labelOnlyStruct:
    img = resize_image_reduce_size(
        cv2.imread(os.path.join("testsetImage", "Images", label[0])),
        512
    )
    imagesTestset[label[0]] = img

In [ ]:
# plt.imshow(imagesTestset["1.jpg"])
# plt.show()

In [ ]:
# # Apply to all image in dataset
for label in labelOnlyStruct:
    imagesTestset[label[0]] = resize_image_reduce_size(imagesTestset[label[0]], size_image_model)

In [ ]:
imagesTestsetArray3=[]
for i in range(len(labelOnlyStruct)):
    image = np.copy(imagesTestset[labelOnlyStruct[i][0]])
    
    # Crop
    cropImages = crop2nimage(image, 3)

    # Resize
    for i in range(len(cropImages)):
        cropImages[i] = cv2.resize(cropImages[i], (size_image_model, size_image_model))

    # Add blur to image
    for i in range(len(cropImages)):
        cropImages[i] = random_blur(cropImages[i], (3, 3))
        
    # Add noise to image
#     for i in range(len(cropImages)):
#         # Randome noice
#         cropImages[i] = random_noise(cropImages[i], 0.1)

    # Debug
#     for i in range(len(cropImages)):
#         plt.imshow(cropImages[i])
#         plt.show()
#     break

    imagesTestsetArray3.append([cropImages[1]])

In [ ]:
print(len(imagesTestsetArray3))

In [ ]:
model_name = "hotaResnet50V2_sigmoid_224x224_v5.keras" # hotaResnet12_sigmoid_128x128_v4 hotaEfficientNetV2S_sigmoid_224x224_v4

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/deep-learning-and-application-group-02", filename=model_name, revision="main", repo_type="model", local_dir="saved_model", local_dir_use_symlinks=False)

In [ ]:
testModel = tf.keras.models.load_model(os.path.join('saved_model', model_name))

In [ ]:
def autoFormatInput2PreditOutput(data):
    return [data[i].reshape((1, size_image_model, size_image_model, 3)) for i in range(len(data))]

def predict2binary(result):
    for i in range(len(result)):
        for j in range(len(result[i])):
            result[i][j] = 1 if result[i][j] >= 0.5 else 0
    return result

def formatImageInput(image, type_format = 0):
    for i in range(len(image)):
        image[i] = np.array(image[i], dtype=np.float32)
        if type_format == 1:
            image[i] = image[i] / np.float32(255.0)
        elif type_format == 2:
            image[i] = image[i] / np.float32(255.0)
            image[i] = image[i] - np.float32(0.5)
            image[i] = image[i] / np.float32(0.5)
    return image
def formatSoftmaxOutput(label):
    for bat in range(len(label)):
        indexOne = np.argmax(label[bat])
        for i in range(len(label[bat])):
            label[bat][i] = 0
        label[bat][indexOne] = 1
    return label

In [ ]:
imagesTestsetArray3[0][0].shape

In [ ]:
image = imagesTestsetArray3[10][0]
plt.imshow(image)
plt.show()
imageData = hota_preprocess_input(image.reshape(([1, *image.shape])))
# autoOnehot2Label(formatSoftmaxOutput(testModel.predict(
#     imageData
# ))[0])
print(predict2binary(testModel.predict(
    imageData
))[0])

In [ ]:
print(testModel.predict(
    imageData
))

In [ ]:
# Build batch for predict
batch_test_size = 64
imagesTestsetArray3Batch = []
for i in range(0, len(imagesTestsetArray3), batch_test_size):
    imagesTestsetArray3Batch_temp = []
    for j in range(i, min(i+batch_test_size, len(imagesTestsetArray3))):
        imagesTestsetArray3Batch_temp.append(
            imagesTestsetArray3[j][0]
        )
    imagesTestsetArray3Batch.append(
        hota_preprocess_input(
            np.array(imagesTestsetArray3Batch_temp, dtype="float32")
        )
    )

In [ ]:
outputOfTestset = []
for i in range(len(imagesTestsetArray3Batch)):
#     labelBatch = formatSoftmaxOutput(testModel.predict(
#         imagesTestsetArray3Batch[i]
#     ))
    labelBatch = predict2binary(testModel.predict(
        imagesTestsetArray3Batch[i]
    ))
    for j in range(len(labelBatch)):
        outputOfTestset.append(list(map(int, labelBatch[j])))
#         outputOfTestset.append(autoOnehot2Label(labelBatch[j]))

In [ ]:
listAddPoint = [
    [0, 2, 3, 4, 5, 8],
    [1, 4, 5, 6, 7, 8, 9],
]
def applyLogicLabel(label):
    # Caculate point
    pointTemp = [0] * len(listAddPoint)
    for i in range(len(label)):
        if int(label[i]) == 0:
            continue
        for j in range(len(listAddPoint)):
            if i in listAddPoint[j]:
                pointTemp[j] += 1
    # Find max
    pointJ, indexJ = 0, 0
    for j in range(len(pointTemp)):
        if pointJ < pointTemp[j]:
            pointJ = pointTemp[j]
            indexJ = j
    # Flter label
    labelRes = [0]*len(label)
    for i in range(len(label)):
        if int(label[i]) == 1 and (i in listAddPoint[indexJ]):
            labelRes[i] = 1
    # Add location
    labelRes[indexJ] = 1
    
    return labelRes

In [ ]:
for i in range(len(labelStruct[12:-2])):
    labelData = labelStruct[i + 12].split(",")
    labelData[1:] = list(map(str, applyLogicLabel(outputOfTestset[i])))
    labelStruct[i + 12] = ",".join(labelData)

In [ ]:
print("\n".join(labelStruct))

In [ ]:
with open("Team_2_Submission_4.txt", "w") as f:
    f.write("\n".join(labelStruct))

In [ ]:
def convertBinVec2LabelList(binVec):
    labels = []
    for i in range(len(binVec)):
        if binVec[i] > 0.5:
            labels.append(labelValue[i])
    return labels

In [ ]:
compareRes12 = """
Anh : A
HoGuom : HG
HoTay : HT
ThapRua : TR
CauTheHuc : CTH
BuuDien : BD
VuonHoa : VH
ChuaTranQuoc : CTQ
DenQuanThanh : DQT
KhachSan : KS
CongVienNuoc : CVN
A,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
167.jpg,1,0,0,1,0,0,0,0,0,0
211.jpg,0,1,0,0,0,0,0,0,0,0
262.jpg,0,1,0,0,0,0,1,0,0,0
248.jpg,1,0,0,0,0,0,0,0,1,0
117.jpg,1,0,1,0,0,0,0,0,0,0
95.jpg,1,0,1,0,0,0,0,0,0,0
127.jpg,0,1,0,0,0,0,0,0,0,0
75.jpg,1,0,1,0,0,0,0,0,0,0
32.jpg,1,0,0,0,0,0,0,0,1,0
227.jpg,1,0,0,0,0,0,0,0,0,0
53.jpg,1,0,0,1,0,0,0,0,0,0
39.jpg,1,0,1,0,0,0,0,0,0,0
296.jpg,0,1,0,0,0,0,0,0,0,0
52.jpg,1,0,1,0,1,0,0,0,0,0
134.jpg,1,0,1,0,0,0,0,0,0,0
37.jpg,1,0,0,0,0,0,0,0,1,0
84.jpg,1,0,0,1,0,0,0,0,0,0
68.jpg,1,0,0,1,0,0,0,0,0,0
16.jpg,1,0,0,0,0,0,0,0,0,0
58.jpg,1,0,0,1,0,0,0,0,0,0
299.jpg,0,1,0,0,0,0,1,0,0,0
153.jpg,1,0,0,0,0,0,0,0,1,0
56.jpg,1,0,0,1,0,0,0,0,0,0
275.jpg,1,0,0,0,0,0,0,0,0,1
43.jpg,1,0,1,0,0,0,0,0,0,0
198.jpg,1,0,1,0,0,0,0,0,0,0
172.jpg,1,0,0,1,0,0,0,0,0,0
283.jpg,1,0,0,1,0,0,0,0,0,0
206.jpg,1,0,0,0,0,0,0,0,0,1
274.jpg,0,1,0,0,0,0,0,0,0,1
209.jpg,0,1,0,0,0,0,0,0,1,0
190.jpg,1,0,0,0,0,0,0,0,0,0
183.jpg,0,1,0,0,0,0,0,0,0,0
54.jpg,1,0,0,0,0,0,0,0,1,0
114.jpg,0,1,0,0,0,0,0,0,0,0
14.jpg,1,0,0,0,0,0,0,0,1,0
236.jpg,1,0,0,0,0,0,0,0,1,0
123.jpg,1,0,0,0,0,0,0,0,1,0
31.jpg,1,0,0,0,0,0,0,0,0,0
122.jpg,0,1,0,0,0,0,0,0,1,1
25.jpg,1,0,0,0,0,0,0,0,1,0
197.jpg,0,1,0,0,0,0,0,0,0,1
10.jpg,1,0,0,1,0,0,0,0,0,0
18.jpg,1,0,0,0,0,0,0,0,0,0
204.jpg,1,0,0,0,0,0,0,0,0,0
231.jpg,1,0,0,1,0,0,0,0,0,0
4.jpg,1,0,0,0,0,0,0,0,1,0
73.jpg,1,0,1,1,0,0,0,0,0,0
77.jpg,1,0,1,0,0,0,0,0,0,0
50.jpg,1,0,1,1,0,0,0,0,0,0
171.jpg,0,1,0,0,0,0,0,0,1,0
163.jpg,1,0,0,1,0,0,0,0,0,0
189.jpg,1,0,0,0,0,0,0,0,0,0
81.jpg,1,0,1,0,0,0,0,0,0,0
221.jpg,1,0,0,0,0,0,1,0,0,0
91.jpg,1,0,1,0,0,0,0,0,0,0
12.jpg,1,0,0,0,0,0,0,0,0,0
101.jpg,1,0,0,1,0,0,0,0,0,0
300.jpg,0,1,0,0,0,0,0,0,0,0
170.jpg,1,0,0,0,0,0,0,0,1,0
162.jpg,1,0,0,0,0,0,0,0,0,0
240.jpg,1,0,0,0,0,0,0,0,1,0
102.jpg,0,1,0,0,0,0,0,0,1,0
181.jpg,0,1,0,0,0,1,0,0,0,0
241.jpg,1,0,0,0,0,0,0,0,1,0
203.jpg,1,0,0,0,0,0,1,0,0,0
200.jpg,0,1,0,0,0,0,0,0,0,0
108.jpg,0,1,0,0,0,0,0,0,1,0
279.jpg,1,0,0,1,0,0,0,0,0,0
193.jpg,1,0,0,0,0,1,0,0,0,0
115.jpg,1,0,1,0,0,0,0,0,0,0
270.jpg,0,1,0,0,0,0,0,0,0,0
182.jpg,1,0,0,0,0,0,0,0,0,0
169.jpg,1,0,0,0,0,0,0,0,1,0
124.jpg,0,1,0,0,0,0,0,0,1,0
254.jpg,1,0,0,0,0,0,0,0,1,0
126.jpg,1,0,0,1,0,0,0,0,0,0
247.jpg,1,0,0,0,0,0,0,0,1,0
136.jpg,1,0,0,0,0,0,0,0,0,0
184.jpg,1,0,1,0,0,0,0,0,0,0
119.jpg,0,1,0,0,0,0,0,0,0,0
71.jpg,1,0,1,0,0,0,0,0,0,0
237.jpg,0,1,0,0,0,0,0,0,1,0
62.jpg,1,0,1,0,0,0,0,0,0,0
6.jpg,1,0,0,1,0,0,0,0,0,0
159.jpg,1,0,0,0,0,0,1,0,0,0
99.jpg,1,0,0,1,0,0,0,0,0,0
48.jpg,1,0,1,0,0,0,0,0,0,0
264.jpg,0,1,0,0,0,0,1,0,0,0
85.jpg,1,0,1,0,0,0,0,0,0,0
79.jpg,1,0,1,0,0,0,0,0,0,0
142.jpg,1,0,0,0,0,0,0,0,1,0
194.jpg,0,1,0,0,0,0,0,0,1,0
90.jpg,1,0,0,0,0,0,0,0,1,0
111.jpg,0,1,0,0,0,0,0,0,0,0
106.jpg,1,0,0,1,0,0,0,0,0,0
202.jpg,1,0,1,0,0,0,0,0,0,0
277.jpg,0,1,0,0,0,0,0,0,0,1
35.jpg,1,0,0,0,0,0,0,0,1,0
179.jpg,0,1,0,0,0,0,0,0,1,0
199.jpg,0,1,0,0,0,0,0,0,0,1
185.jpg,0,0,0,0,0,0,0,0,0,0
233.jpg,1,0,0,1,0,0,0,0,0,0
220.jpg,0,1,0,0,0,0,1,0,0,0
188.jpg,1,0,1,0,0,0,0,0,0,0
96.jpg,1,0,1,0,0,0,0,0,0,0
186.jpg,1,0,1,0,0,1,0,0,0,0
278.jpg,0,1,0,0,0,0,0,0,0,0
78.jpg,1,0,1,0,0,0,0,0,0,0
11.jpg,1,0,0,0,0,0,0,0,1,0
157.jpg,1,0,0,1,0,0,0,0,0,0
286.jpg,1,0,0,0,0,0,0,0,0,1
155.jpg,1,0,0,0,0,0,0,0,0,0
110.jpg,0,1,0,0,0,0,0,0,1,0
168.jpg,1,0,1,0,0,0,0,0,0,0
173.jpg,1,0,0,1,0,0,0,0,0,0
139.jpg,0,1,0,0,0,0,0,0,1,0
30.jpg,1,0,0,0,0,0,0,0,1,0
282.jpg,1,0,0,0,0,0,0,0,0,1
276.jpg,0,1,0,0,0,0,0,0,0,1
218.jpg,1,0,0,1,0,0,0,0,0,0
29.jpg,1,0,1,0,0,0,0,0,0,0
294.jpg,0,1,0,0,0,0,0,0,0,1
284.jpg,1,0,0,1,0,0,0,0,0,0
267.jpg,1,0,0,1,0,0,0,0,0,0
261.jpg,0,1,0,0,0,0,0,0,0,0
290.jpg,0,1,0,0,0,0,1,0,0,0
137.jpg,1,0,0,1,0,0,0,0,0,0
239.jpg,1,0,0,0,0,0,0,0,1,0
15.jpg,1,0,0,0,0,0,0,0,0,0
251.jpg,1,0,0,0,0,0,0,0,1,0
55.jpg,1,0,0,1,0,0,0,0,0,0
297.jpg,0,1,0,0,0,0,0,0,0,1
100.jpg,1,0,0,1,0,0,0,0,0,0
210.jpg,0,1,0,0,0,0,0,0,1,0
59.jpg,1,0,0,1,0,0,0,0,0,0
255.jpg,1,0,0,0,0,0,0,0,1,0
19.jpg,1,0,0,0,0,0,0,0,0,0
24.jpg,1,0,0,1,0,0,0,0,0,0
86.jpg,1,0,1,0,0,0,0,0,0,0
217.jpg,1,0,0,0,0,0,1,0,0,0
128.jpg,0,1,0,0,0,0,0,0,0,0
228.jpg,1,0,0,0,0,0,1,0,1,0
28.jpg,1,0,0,0,0,0,0,0,0,0
195.jpg,1,0,1,0,0,0,0,0,0,0
288.jpg,1,0,0,1,0,0,0,0,0,0
152.jpg,1,0,0,0,0,0,0,0,0,0
116.jpg,0,1,0,0,0,0,1,0,0,0
36.jpg,1,0,0,1,0,0,0,0,0,0
191.jpg,1,0,0,1,0,0,0,0,0,0
9.jpg,1,0,0,1,0,0,0,0,0,0
70.jpg,1,0,1,0,0,0,0,0,0,0
201.jpg,1,0,1,0,0,0,0,0,0,0
46.jpg,1,0,1,1,0,0,0,0,0,0
13.jpg,1,0,0,0,0,0,0,0,0,0
207.jpg,1,0,1,0,0,0,0,0,0,0
291.jpg,0,1,0,0,0,0,1,0,0,0
107.jpg,1,0,0,1,0,0,0,0,0,0
252.jpg,1,0,0,0,0,0,0,0,1,0
226.jpg,1,0,0,0,0,0,0,0,0,0
76.jpg,1,0,1,1,0,0,0,0,0,0
223.jpg,0,1,0,0,0,0,0,0,0,0
135.jpg,0,1,0,0,0,0,0,0,1,0
271.jpg,0,1,0,0,0,0,0,0,0,0
246.jpg,1,0,0,0,0,0,0,0,1,0
20.jpg,1,0,0,1,0,0,0,0,0,0
44.jpg,1,0,0,0,0,0,1,0,1,0
269.jpg,1,0,0,0,0,0,0,0,0,0
129.jpg,1,0,1,0,0,0,0,0,0,0
97.jpg,1,0,1,0,0,0,0,0,0,0
280.jpg,0,1,0,0,0,0,0,0,0,0
21.jpg,1,0,0,0,0,0,0,0,1,0
63.jpg,1,0,0,1,0,0,0,0,0,0
238.jpg,1,0,0,0,0,0,0,0,1,0
8.jpg,1,0,0,1,0,0,0,0,0,0
272.jpg,0,1,0,0,0,0,0,0,1,0
225.jpg,1,0,0,0,0,0,0,0,0,0
177.jpg,0,1,0,0,0,0,0,0,0,0
141.jpg,1,0,0,0,0,0,0,0,1,0
287.jpg,0,1,0,0,0,0,0,0,0,1
212.jpg,1,0,0,1,0,0,0,0,0,0
219.jpg,1,0,0,0,0,0,0,0,1,0
113.jpg,0,1,0,0,0,1,0,0,0,0
74.jpg,1,0,0,1,0,0,0,0,0,0
41.jpg,1,0,0,1,0,0,0,0,0,0
89.jpg,1,0,1,0,0,0,0,0,0,0
38.jpg,1,0,0,0,0,0,0,0,1,0
103.jpg,1,0,0,1,0,0,0,0,0,0
26.jpg,1,0,0,0,0,0,0,0,0,0
256.jpg,1,0,0,0,0,0,0,0,1,0
222.jpg,1,0,0,0,0,0,1,0,0,0
285.jpg,1,0,0,1,0,0,0,0,0,0
61.jpg,1,0,0,1,0,0,0,0,0,0
149.jpg,0,1,0,0,0,0,0,0,0,0
64.jpg,1,0,0,1,0,0,0,0,0,0
259.jpg,1,0,0,0,0,0,0,0,1,0
131.jpg,1,0,1,0,0,0,0,0,0,0
215.jpg,1,0,0,1,0,0,0,0,0,0
281.jpg,1,0,0,1,0,0,0,0,0,0
66.jpg,1,0,0,1,0,0,0,0,0,0
27.jpg,1,0,0,0,0,0,0,0,1,0
57.jpg,1,0,0,1,0,0,0,0,0,0
146.jpg,1,0,0,0,0,0,0,0,1,0
143.jpg,1,0,0,0,0,0,0,0,1,0
174.jpg,1,0,0,1,0,0,1,0,0,0
295.jpg,0,1,0,0,0,0,0,0,0,0
244.jpg,1,0,0,0,0,0,0,0,1,0
234.jpg,1,0,0,0,0,0,0,0,0,0
257.jpg,1,0,0,0,0,0,0,0,0,0
258.jpg,1,0,0,0,0,0,0,0,0,0
65.jpg,0,1,0,0,0,0,0,0,1,0
112.jpg,1,0,0,0,0,1,0,0,0,0
205.jpg,1,0,0,0,0,0,0,0,0,0
130.jpg,1,0,0,0,0,0,0,0,0,0
144.jpg,0,1,0,0,0,0,0,0,0,0
140.jpg,1,0,0,0,0,0,0,0,1,0
156.jpg,1,0,0,0,0,0,0,0,1,0
92.jpg,1,0,1,0,0,0,0,0,0,0
94.jpg,1,0,1,0,0,0,0,0,0,0
132.jpg,0,1,0,0,0,0,0,0,0,0
83.jpg,1,0,0,1,0,0,0,0,0,0
148.jpg,0,1,0,0,0,0,0,0,0,0
253.jpg,1,0,0,0,0,0,0,0,1,0
175.jpg,1,0,0,1,0,0,0,0,0,0
104.jpg,1,0,0,1,0,0,0,0,0,0
216.jpg,1,0,0,0,0,0,1,0,0,0
87.jpg,1,0,1,0,0,0,0,0,0,0
292.jpg,0,1,0,0,0,0,0,0,0,1
23.jpg,1,0,0,0,0,0,0,0,0,0
161.jpg,0,1,0,0,0,0,1,0,0,0
88.jpg,1,0,0,1,0,0,0,0,0,0
49.jpg,1,0,1,0,0,0,0,0,0,0
33.jpg,1,0,0,0,0,0,0,0,0,0
164.jpg,1,0,0,1,0,0,0,0,0,0
165.jpg,1,0,0,0,0,0,1,0,0,0
105.jpg,1,0,0,0,0,0,0,0,1,0
80.jpg,1,0,0,1,0,0,0,0,0,0
98.jpg,1,0,0,1,0,0,0,0,0,0
178.jpg,0,1,0,0,0,0,0,0,0,0
263.jpg,0,1,0,0,0,0,1,0,0,0
17.jpg,1,0,0,0,0,0,0,0,1,0
245.jpg,1,0,0,0,0,0,0,0,1,0
232.jpg,1,0,0,1,0,0,0,0,0,0
133.jpg,0,1,0,0,0,0,0,0,1,0
243.jpg,1,0,0,0,0,0,0,0,1,0
82.jpg,1,0,0,1,0,0,0,0,0,0
42.jpg,1,0,1,0,0,0,0,0,0,0
34.jpg,1,0,0,0,0,0,0,0,1,0
22.jpg,1,0,0,0,0,0,0,0,0,0
298.jpg,0,1,0,0,0,0,0,0,0,0
3.jpg,1,0,0,1,0,0,0,0,0,0
176.jpg,0,1,0,0,0,0,0,0,0,0
230.jpg,1,0,0,1,0,0,0,0,0,0
158.jpg,1,0,0,0,0,0,0,0,0,0
150.jpg,0,1,0,0,0,0,0,0,0,0
160.jpg,0,1,0,0,0,0,1,0,0,0
147.jpg,1,0,0,0,0,0,0,0,1,0
72.jpg,1,0,1,0,0,0,0,0,0,0
268.jpg,1,0,0,0,0,0,0,0,0,0
60.jpg,1,0,0,1,0,0,0,0,0,0
187.jpg,1,0,0,1,0,0,0,0,0,0
260.jpg,1,0,0,0,0,0,0,0,1,0
224.jpg,1,0,0,0,0,0,1,0,0,0
93.jpg,1,0,1,0,0,0,0,0,0,0
166.jpg,1,0,0,1,0,0,0,0,0,0
196.jpg,0,1,0,0,0,0,0,0,0,0
1.jpg,1,0,0,0,0,0,0,0,1,0
109.jpg,0,1,0,0,0,0,1,0,0,0
180.jpg,0,1,0,0,0,0,0,0,0,0
40.jpg,1,0,1,1,0,0,0,0,0,0
69.jpg,1,0,0,1,0,0,0,0,0,0
138.jpg,1,0,0,0,0,0,0,0,0,0
229.jpg,1,0,0,1,0,0,0,0,0,0
145.jpg,1,0,0,0,0,0,0,0,1,0
235.jpg,1,0,0,0,0,0,0,0,0,0
7.jpg,1,0,0,1,0,0,0,0,0,0
67.jpg,1,0,0,1,0,0,0,0,0,0
214.jpg,1,0,0,0,0,0,0,0,1,0
45.jpg,1,0,0,1,0,0,0,0,0,0
51.jpg,1,0,1,0,0,0,0,0,0,0
250.jpg,1,0,0,0,0,0,0,0,1,0
121.jpg,1,0,0,0,0,0,0,0,1,0
208.jpg,0,1,0,0,0,0,0,0,0,0
47.jpg,1,0,0,1,0,0,0,0,0,0
5.jpg,1,0,0,1,0,0,0,0,0,0
154.jpg,0,1,0,0,0,0,0,0,0,0
151.jpg,1,0,1,0,0,0,0,0,0,0
2.jpg,1,0,1,1,0,0,0,0,0,0
249.jpg,0,1,0,0,0,0,0,0,0,0
289.jpg,1,0,0,1,0,0,0,0,0,0
273.jpg,0,1,0,0,0,0,0,0,0,0
192.jpg,1,0,0,1,0,0,0,0,0,0
118.jpg,1,0,0,0,0,0,0,0,0,0
213.jpg,1,0,0,1,0,0,0,0,0,0
265.jpg,0,1,0,0,0,0,0,0,0,0
125.jpg,1,0,0,1,0,0,0,0,0,0
242.jpg,1,0,0,0,0,0,0,0,1,0
266.jpg,0,1,0,0,0,0,0,0,0,1
293.jpg,0,1,0,0,0,0,0,0,0,1
120.jpg,1,0,0,0,0,0,0,0,1,0
A,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
"""

In [ ]:
compareRes12Arr = compareRes12[1:-1].split("\n")
print(len(compareRes12Arr))

In [ ]:
import random

In [ ]:
linetestcase = random.randrange(12, 312)
print(linetestcase + 1)
testcase = linetestcase-12
print(applyLogicLabel(outputOfTestset[testcase]))
print(convertBinVec2LabelList(applyLogicLabel(outputOfTestset[testcase])))
print(convertBinVec2LabelList(list(map(int, compareRes12Arr[linetestcase].split(",")[1:]))))
plt.imshow(imagesTestsetArray3[testcase][0])
plt.show()
# print(autoOnehot2Label(formatSoftmaxOutput(testModel.predict(
#     hota_preprocess_input(
#         np.array(imagesTestsetArray3[testcase], dtype="float32")
#     )
# ))[0]))
# print(predict2binary(testModel.predict(
#     hota_preprocess_input(
#         np.array(imagesTestsetArray3[testcase], dtype="float32")
#     )
# ))[0])

In [ ]:
"""
175: res12
"""